# Final Assessment
**Mohoh dibaca terlebih dahulu:**
* Dalam notebook ini anda akan mengerjakan assessment **acara 7.**
* Anda bebas menulis script/kode yang diinginkan selama itu mengasilkan hasil yang sama dengan clue jawaban. Clue jawaban berupa gambar yang terlampir pada tiap cell dibawah cell kode yang anda tulis.
* Gambar yang ada dalam notebook ini merupakan hasil yang diharapkan muncul ketika anda berhasil menulisakan kode yang benar. Gambar ini juga menjadi patokan output kode anda, Sehingga nantinya akan muncul 2 gambar yang serupa ketika anda berhasil running kode yang benar.
* Silahkan menuliskan kode atau kata pada tanda "(?)"
* Silahkan menjawab pertanyaan yang ada dalam notebook ini lansung pada cell yang disediakan.
* Selamat mengerjakan

In [ ]:
#cek lokasi directory kerja
#perhatikan letak data dan file input pada directory
#jika lokasi data dan file berbeda dengan directory kerja, 
#maka pastikan bahwa anda mengakses file dan data yang berada pada directory yang berbeda
#cara lainnya adalah memindahkan data dan file input ke directory kerja
!pwd

## Acara 7 Machine Learning in Bionformatics

- Dataset: **SERI A Data Praktikum Metabolomic.xlsx - NMR data Blast.csv** dalam directory **data**
- Metadata:
    - Status = Tahan & rentan
    
**Pertanyaan**
- Ada berapa sample, feature, dan dimensi pada dataset ini?

In [ ]:
# Load Library
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
# load data
raw =  (?)#load disni #!!!
raw (?) #print 5 data awal dengan index mulai dari 0

In [ ]:
#Print nama column yang ada dalam DataFrame raw
#raw.columns 

### Standarize the data and EDA

In [ ]:
# Data yang memiliki pencilan akan distandarisasi untuk meminimalkan range data dan pencilan
raw_drop = raw.drop(columns=[(?)]) #drop/menghilankan column tertentu

#Standarisasi
scaler = preprocessing.StandardScaler()
scaler.fit((?))
raw_scaled_array = scaler.transform((?))
raw_scaled = pd.DataFrame(raw_scaled_array, columns = raw.columns[2:19])

#merging 2 DataFrame menjadi 1 DataFrame
merge_raw = pd.concat([raw[['Sample','Status']], raw_scaled], axis=1) #merged 2 dataframe
merge_raw. (?) #print 5 data awal

In [ ]:
#Perbandingan data summary sebelum dan sesudah standarisasi
#display(raw.describe()) #hapus tanda "#" untuk running kode display()
#display(merge_raw.describe())

In [ ]:
#Indexing, mengubah Column Sample menjadi index
#Bandingkan antara merge_raw dengan indexed. Perhatikan column paling kiri/index
indexed = merge_raw. (?)
indexed.head()

![7.4_print_standarized_indexed_head](pictures/7.4_print_standarized_indexed_head.PNG)

In [ ]:
#make a heatmap after standardization
sns.set_theme()
sns.heatmap(indexed.drop(columns=(?)))
plt.show()

![7.5_heatmap](pictures/7.5_heatmap.PNG)

### PCA

In [ ]:
# split data into features (X) and labels (y). 
# features (X) disini merujuk pada value dari masing-masing senyawa 
# labels (Y) merujuk pada Status tiap sample
X = merge_raw.values
Y = raw[['Sample','Status']].values #print value column Sample dan Status

In [ ]:
seed = 0
ndimensions = (?) #tuliskan berapa banyak dimensi yang ingin dibuat

pca = PCA(n_components=ndimensions, random_state=seed)
pca.fit(raw_scaled)
X_pca_array = pca.transform(raw_scaled)
X_pca = pd.DataFrame(X_pca_array, columns=(?)) # PC=principal component
X_pca.head()

In [ ]:
nclusters = (?) #this is the k in kmeans. 
#Terhgantung berapa banyak kelompok yang ingin dibuat. Clue: Perhatikan berapa banyak status

km = KMeans(n_clusters=nclusters, random_state=seed)
km.fit(raw_scaled)

# predict the cluster for each data point
y_cluster_kmeans = km.predict(raw_scaled)
y_cluster_kmeans

In [ ]:
Y2 = pd.Categorical(merge_raw['Status'].values).codes #!!! #Mengubah string/kata menjadi angka
y_id_array2 = Y2

df_plot = X_pca.copy()
df_plot['ClusterKmeans'] = y_cluster_kmeans
df_plot['Status'] = y_id_array2 # also add actual labels so we can use it in later plots

df_plot.head()

In [ ]:
def plotData(df, groupby):
    "make a scatterplot of the first two principal components of the data, colored by the groupby field"
    
    # make a figure with just one subplot.
    # you can specify multiple subplots in a figure, 
    # in which case ax would be an array of axes,
    # but in this case it'll just be a single axis object.
    fig, ax = plt.subplots(figsize = (7,7))

    # color map
    cmap = mpl.cm.get_cmap('prism')

    # we can use pandas to plot each cluster on the same graph.
    # see http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html
    for i, cluster in df.groupby(groupby):
        cluster.plot(ax = ax, # need to pass this so all scatterplots are on same graph
                     kind = 'scatter', 
                     x = 'PC1', y = 'PC2',
                     color = cmap(i/(nclusters-1)), # cmap maps a number to a color
                     label = "%s %i" % (groupby, i), 
                     s=30) # dot size
    ax.grid()
    ax.axhline(0, color='black')
    ax.axvline(0, color='black')
    ax.set_title("Principal Components Analysis");

In [ ]:
# plot the clusters each datapoint was assigned to
plotData(df_plot, 'ClusterKmeans')
plotData(df_plot, 'Status')

![7.8_PCA_plotcluster](pictures/7.8_PCA_plotcluster.PNG)
![7.8_PCA_plotstatus](pictures/7.8_PCA_plotstatus.PNG)

**Pertanyaan**
* Berdasarkan PCA K-means, berapa banyak sample yang memiliki fenotip tahan?
* Bandingkan antara PCA K-means dengan PCA status. Apakah terdapat perbedaan? Jika iya, coba interpretasikan hasil kedua PCA yang anda lihat?

**Jawaban**
* Terdapat 3 sampe berfenotip tahan
* Iya, pada PCA K-means terdapat 3 sample berfenotip sedangkan pada PCA-status terdapat 6 sample berfenotip tahan, artinya terdapat 3 sample yang yang mana oleh algoritma K-means menunujukkan dengan sample/label yang sebenarnya, sehingga clustering menggunakan K-means PCA belum cukup mampu membedakan/mengelompokkan sample sesui label/sample. 